### Tutorial for dialog flow framework 
Descriptionn of dff and links....

In [ ]:
# # installation
# !pip install git+https://github.com/deepmipt/dialog_flow_framework.git@dev

If you want to get more info about runtime of dff you can change level of logging

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.DEBUG)
logger = logging.getLogger(__name__)

Let's try a simple example. 

We need keywords `GRAPH`, `RESPONSE`, `TRANSITIONS` for dialog description. `GRAPH` - it's spetial keyword for dialog graph definition, `RESPONSE` - defines answers of our bot, `TRANSITIONS` - defines conditions for transition in a dialog graph.

The next abstractions are `Context` and `Actor`. `Context` containes all context of a dialog. Context is easy serializable by different ways (e.g. `Context().json()`). `Actor` executes handler incomming `Context` and return updated `Context` with selected response. 

In [16]:
from dff import GRAPH, RESPONSE, TRANSITIONS
from dff import Context, Actor

flow = {
    GRAPH: {
        "hello_node": {RESPONSE: "hi", TRANSITIONS: {"im_fine": "how are you", "hello_node": "hello"}},
        "im_fine": {RESPONSE: "i'm fine"},
        "fallback": {RESPONSE: "oops", TRANSITIONS: {"hello_node": "hello"}},
    }
}


ctx = Context()
actor = Actor(
    {"start_flow": flow},
    start_label=("start_flow", "hello_node"),
    fallback_label=("start_flow", "fallback"),
)
in_text = "hello"
ctx.add_request(in_text)
ctx = actor(ctx)
out_text = ctx.last_response
print(f"you: {in_text}")
print(f"bot: {ctx.last_response}")
assert ctx.last_response == "hi"

in_text = "how are you"
ctx.add_request(in_text)
ctx = actor(ctx)
print(f"you: {in_text}")
print(f"bot: {ctx.last_response}")
assert ctx.last_response == "i'm fine"

in_text = "ok"
ctx.add_request(in_text)
ctx = actor(ctx)
print(f"you: {in_text}")
print(f"bot: {ctx.last_response}")
assert ctx.last_response == "oops"


2021-08-04 15:20:47,215 - dff.core.actor - DEBUG - global transitions sorted by priority = []
2021-08-04 15:20:47,219 - dff.core.actor - DEBUG - local transitions sorted by priority = [('start_flow', 'hello_node', 1.0)]
2021-08-04 15:20:47,226 - dff.core.actor - DEBUG - global transitions sorted by priority = []
2021-08-04 15:20:47,231 - dff.core.actor - DEBUG - local transitions sorted by priority = [('start_flow', 'im_fine', 1.0)]
2021-08-04 15:20:47,236 - dff.core.actor - DEBUG - global transitions sorted by priority = []
2021-08-04 15:20:47,237 - dff.core.actor - DEBUG - local transitions sorted by priority = []
you: hello
bot: hi
you: how are you
bot: i'm fine
you: ok
bot: oops
